In [ ]:
# import library ที่จำเป็น
import pandas as pd
import numpy as np

In [ ]:
# จาก tutorial

from ase.io import read
PI = np.pi
def get_box_parameters(cifname):
    """
    ดึงข้อมูลพารามิเตอร์ของ Unit Cell ของ MOF

    cifname : CIF file
    """
    content = open(cifname, 'r').readlines()
    for line in content:
        if "_cell_length_a" in line:
            a = float(line.split()[1])
        if "_cell_length_b" in line:
            b = float(line.split()[1])
        if "_cell_length_c" in line:
            c = float(line.split()[1])
        if "_cell_angle_alpha" in line:
            alpha_r = float(line.split()[1]) * PI / 180.0
        if "_cell_angle_beta" in line:
            beta_r = float(line.split()[1]) * PI / 180.0
        if "_cell_angle_gamma" in line:
            gamma_r = float(line.split()[1]) * PI / 180.0

    return a, b, c, alpha_r, beta_r, gamma_r

def get_box_matrix(a, b, c, alpha_r, beta_r, gamma_r):
    """
    คำนวณ Unit Cell Matrix 

    cifname : CIF file
    """

    a_x = a
    a_y = 0.0
    a_z = 0.0
    b_x = b * np.cos(gamma_r)
    b_y = b * np.sin(gamma_r)
    b_z = 0.0
    c_x = c * np.cos(beta_r)
    c_y = (b * c * np.cos(alpha_r) - b_x * c_x) / b_y
    c_z = np.sqrt(c**2 - c_x**2 - c_y**2)

    cell_matrix = np.matrix([
        [a_x, a_y, a_z], 
        [b_x, b_y, b_z], 
        [c_x, c_y, c_z]], 
        dtype=float)

    crs = np.cross([b_x, b_y, b_z], [c_x, c_y, c_z])
    inverse_matrix = cell_matrix.I
    unit_cell_volume = a_x * crs[0] + a_y * crs[1] + a_z * crs[2]
    
    return cell_matrix, inverse_matrix, unit_cell_volume

def get_atomic_coordinates(cifname):
    """
    ดึงข้อมูล atomic symbol และ Cartesian coordinates
    """
    ase_atoms = read(cifname) 
    return ase_atoms.get_chemical_symbols(), ase_atoms.get_positions()

In [ ]:
# อ่านไฟล์ train.csv
data_trian = pd.read_csv('train.csv')

mof_train = []
for i in data_trian['MOFname']:
    # นำ MOFname ใน train.csv มาอ่านไฟล์ cif และ extract features ต่างๆ ออกมา
    cell = list(get_box_parameters('mof_cif_train/'+i+'.cif'))
    el, coords = get_atomic_coordinates('mof_cif_train/'+i+'.cif')
    cell_matrix, inverse_matrix, unit_cell_volume = get_box_matrix(*cell)
    mof_test.append({
        'name':i,
        'cell':cell, 
        'coords':coords,
        'elements':el,
        'cell_matrix':cell_matrix,
        'inverse_matrix':inverse_matrix,
        'unit_cell_volume':unit_cell_volume,
    })

# บันทึกผลลัพธ์ train structure ไว้ใช้งาน 
np.save('train_structures', mof_train, allow_pickle=True)

In [ ]:
# อ่านไฟล์ test.csv
data_test = pd.read_csv('test.csv')

mof_test = []
for i in data_test['MOFname']:
    # นำ MOFname ใน test.csv มาอ่านไฟล์ cif และ extract features ต่างๆ ออกมา    
    cell = list(get_box_parameters('mof_cif_test/'+i+'.cif'))
    el, coords = get_atomic_coordinates('mof_cif_test/'+i+'.cif')
    cell_matrix, inverse_matrix, unit_cell_volume = get_box_matrix(*cell)
    mof_test.append({
        'name':i,
        'cell':cell, 
        'coords':coords,
        'elements':el,
        'cell_matrix':cell_matrix,
        'inverse_matrix':inverse_matrix,
        'unit_cell_volume':unit_cell_volume,
    })

# บันทึกผลลัพธ์ของ test structure ไว้ใช้งาน     
np.save('test_structures', mof_test, allow_pickle=True)